# Lesson 1: Your first agent with Amazon Bedrock

## Preparation 
<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> and other files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix - Tips and Help"</em> Lesson.</p>

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
roleArn=os.getenv('BEDROCKAGENTROLE')

In [2]:
# Before you start, please run the following code to set up your environment.
# This code will reset the environment (if needed) and prepare the resources for the lesson.
# It does this by quickly running through all the code from the previous lessons.

!sh ./ro_shared_data/reset.sh

roleArn=os.getenv('BEDROCKAGENTROLE')

Resetting environment (if nessesary)
Agent reset process completed.
Lambda reset process completed.
Guardrail reset process completed.
Environment reset complete.


## Start of the lesson

In [3]:
import boto3

In [4]:
bedrock_agent = boto3.client(service_name='bedrock-agent', region_name='us-east-1')

In [5]:
create_agent_response = bedrock_agent.create_agent(
    agentName='mugs-customer-support-agent',
    foundationModel='anthropic.claude-3-haiku-20240307-v1:0',
    instruction="""You are an advanced AI agent acting as a front line customer support agent.""",
    agentResourceRoleArn=roleArn
)

AccessDeniedException: An error occurred (AccessDeniedException) when calling the CreateAgent operation: User: arn:aws:iam::092413168457:user/openailab is not authorized to perform: bedrock:CreateAgent

In [ ]:
create_agent_response

In [ ]:
!ls ..
!tar -cvf all_files.tar *

In [ ]:
agentId = create_agent_response['agent']['agentId']

In [ ]:
from helper import *

In [ ]:
wait_for_agent_status(
    agentId=agentId, 
    targetStatus='NOT_PREPARED'
)

In [ ]:
bedrock_agent.prepare_agent(
    agentId=agentId
)

In [ ]:
wait_for_agent_status(
    agentId=agentId, 
    targetStatus='PREPARED'
)

In [ ]:
create_agent_alias_response = bedrock_agent.create_agent_alias(
    agentId=agentId,
    agentAliasName='MyAgentAlias',
)

agentAliasId = create_agent_alias_response['agentAlias']['agentAliasId']

wait_for_agent_alias_status(
    agentId=agentId,
    agentAliasId=agentAliasId,
    targetStatus='PREPARED'
)

In [ ]:
bedrock_agent_runtime = boto3.client(service_name='bedrock-agent-runtime', region_name='us-west-2')

In [ ]:
import uuid

In [ ]:
message = "Hello, I bought a mug from your store yesterday, and it broke. I want to return it."

sessionId = str(uuid.uuid4())

invoke_agent_response = bedrock_agent_runtime.invoke_agent(
    agentId=agentId,
    agentAliasId=agentAliasId,
    inputText=message,
    sessionId=sessionId,
    endSession=False,
    enableTrace=True,
)

In [ ]:
event_stream = invoke_agent_response["completion"]

In [ ]:
for event in event_stream:
    print(event)

In [ ]:
message = "Hello, I bought a mug from your store yesterday, and it broke. I want to return it."

sessionId = str(uuid.uuid4())

In [ ]:
invoke_agent_and_print(
    agentAliasId=agentAliasId,
    agentId=agentId,
    sessionId=sessionId,
    inputText=message,
    enableTrace=True,
)